In [ ]:
import pandas as pd

In [ ]:
using_collab = True

if using_collab :
  from google.colab import drive
  drive.mount('/content/drive')

file_path = "/content/drive/MyDrive" #file_path를 설정해주세요.

Mounted at /content/drive


In [ ]:
rawdata1= pd.read_excel(file_path +'/KISTI/rawdata/아마존_수집정보_20230328 (1).xlsx',sheet_name=[1])
rawdata2= pd.read_excel(file_path +'/KISTI/rawdata/amazon_data.xlsx',sheet_name=[1,2,3,4,5])

In [ ]:
rawdata1[1].head(5)

,LEVEL1,LEVEL2,LEVEL3,LEVEL4,url,product_cnt,modelling
0,Appliances,Appliance Warranties,Appliance Warranties,Appliance Warranties,https://www.amazon.com/Best-Sellers-Appliances...,1,O
1,Appliances,Cooktops,Cooktops,Cooktops,https://www.amazon.com/Best-Sellers-Appliances...,100,O
2,Appliances,Dishwashers,Built-In Dishwashers,Built-In Dishwashers,https://www.amazon.com/Best-Sellers-Appliances...,96,O
3,Appliances,Dishwashers,Countertop Dishwashers,Countertop Dishwashers,https://www.amazon.com/Best-Sellers-Appliances...,96,O
4,Appliances,Dishwashers,Portable Dishwashers,Portable Dishwashers,https://www.amazon.com/Best-Sellers-Appliances...,56,O


In [ ]:
rawdata1[1].groupby(['LEVEL1','LEVEL2','LEVEL3']).size().reset_index()[['LEVEL1','LEVEL2','LEVEL3']] #,,'LEVEL4'

,LEVEL1,LEVEL2,LEVEL3
0,Appliances,Appliance Warranties,Appliance Warranties
1,Appliances,Cooktops,Cooktops
2,Appliances,Dishwashers,Built-In Dishwashers
3,Appliances,Dishwashers,Countertop Dishwashers
4,Appliances,Dishwashers,Portable Dishwashers
...,...,...,...
2392,Toys & Games,Stuffed Animals & Toys,Plush Puppets
2393,Toys & Games,Stuffed Animals & Toys,Plush Purses
2394,Toys & Games,Stuffed Animals & Toys,Plush Toy & Blanket Sets
2395,Toys & Games,Stuffed Animals & Toys,Stuffed Animal Clothing & Accessories


In [ ]:
rawdata1[1].groupby(['LEVEL1','LEVEL2','LEVEL3','LEVEL4']).size().reset_index()[['LEVEL1','LEVEL2','LEVEL3','LEVEL4']].to_csv("/content/drive/MyDrive/KISTI/rawdata/amazon_level1234.csv")

In [ ]:
is_hierarchy_consistent = rawdata1[1].groupby(['LEVEL1','LEVEL2','LEVEL3','LEVEL4']).size().reset_index().duplicated(subset=['LEVEL1', 'LEVEL2', 'LEVEL3','LEVEL4'], keep=False)

print(rawdata1[1].groupby(['LEVEL1','LEVEL2','LEVEL3','LEVEL4']).size().reset_index()[is_hierarchy_consistent])


Empty DataFrame
Columns: [LEVEL1, LEVEL2, LEVEL3, LEVEL4, 0]
Index: []


In [ ]:
len(rawdata1[1])

10985

In [ ]:
rawdata1 = rawdata1[1]

In [ ]:
rawdata2[3].head(5)

,url,product_key,product_idx,create_date,detail_table_key,detail_table_value
0,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,1,2022-12-02,Product Dimensions,3.5 x 3.8 x 3.7 inches; 8.78 Ounces
1,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,2,2022-12-02,Item model number,W1681
2,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,3,2022-12-02,Date First Available,2019-09-17 00:00:00
3,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,4,2022-12-02,Manufacturer,MMIZOO
4,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,5,2022-12-02,ASIN,B07XYW52VQ


In [ ]:
len(rawdata2[1])

584

In [ ]:
len(rawdata2[2])

2233

In [ ]:
len(rawdata2[3])

6475

In [ ]:
len(rawdata2[4])

3253

In [ ]:


len(rawdata2[5])

584

## Task 1 : 제품 설명 -> 4단계 라벨 카테고리 생성

> 들여쓴 블록



### 예시  
**instruction**  
Classify the product into four levels of categories based on the following product name and description.

<s>다음 제품 이름과 설명을 기반으로 상품의 카테고리를 4단계로 분류하세요.</s>

**input**  
The product name is [product_name] and the body of content of this product is [body_content].
How can this product be categorized into four levels?


**output**  
Lv 1 : [level1]; lv2 : [level2], lv3 : [level3], lv 4 : [level4]

In [ ]:
#instruction #context #response #category

task1_data = pd.DataFrame()

# Generate 'context' column
# input level 3 일 때
task1_data['context'] = "The product name is " + rawdata2[5]['product_name']\
                        + ' and the body of content of this product is '  \
                        + rawdata2[1]["body_content"]


# Generate 'response' column
task1_data['response'] = "The following products can be categorized into four hierarchical levels: "+'level 1: ' + rawdata2[5]['level1']\
                        + ' level 2: ' + rawdata2[5]['level2']\
                        + ' level 3: ' + rawdata2[5]['level3'] \
                        + ' level 4: ' + rawdata2[5]['level4']


task1_data['instruction'] = "Classify the product into four levels of categories based on the following product name and description."#"다음 제품 이름과 설명을 기반으로 상품의 카테고리를 4단계로 분류하세요."
task1_data['category'] = "task1"

task1_data['index'] = task1_data.index

task1_data['data'] = "amazon2-5"

task1_data['product_key'] = "category1"


task1_data = task1_data[['instruction','context','response','category','index','data','product_key']]

task1_data.tail()

,instruction,context,response,category,index,data,product_key
579,Classify the product into four levels of categ...,"The product name is XS Power Li1215 12 Volt, 1...",The following products can be categorized into...,task1,579,amazon2-5,category1
580,Classify the product into four levels of categ...,"The product name is BMK Car Battery Charger, 1...",The following products can be categorized into...,task1,580,amazon2-5,category1
581,Classify the product into four levels of categ...,The product name is CRAFTSMAN GOLF 15 AMP 48 V...,The following products can be categorized into...,task1,581,amazon2-5,category1
582,Classify the product into four levels of categ...,The product name is 67.2V 5A Charger 60V Smart...,The following products can be categorized into...,task1,582,amazon2-5,category1
583,Classify the product into four levels of categ...,The product name is BatteryMINDer 244CEC2-AA-S...,The following products can be categorized into...,task1,583,amazon2-5,category1


In [ ]:
task1_data['context'][579]

'The product name is XS Power Li1215 12 Volt, 15 AMP Lithium Ion Battery Charger and the body of content of this product is [`XS Power Li1215 12V Lithium Ion Battery Charger 15A The Li1215 is an advanced battery charger designed specifically for high performance 12V lithium ion batteries. This charger combines the reliability of a High Frequency MOSFET rectified assembly with an advanced microprocessor control board to create a battery charger that can rapidly and safely recharge 12V lithium ion batteries in several sizes.`]'

In [ ]:
len(task1_data)

584

## Task 2-1 - 상위, 하위 카테고리 예측 (Lv3)



### 예시  
**instruction**.
Predict higher-level categories and generate lower-level categories using the provided values.  
<s>주어진 값을 이용하여 상위 카테고리를 예측하고 하위 카테고리를 생성하시오. </s>

**input**  
Lv1:_    ; Lv2:_    ; Lv3: Consoles; Lv4:_


**output**  
lv1: Video Games; lv2: Xbox Series X ; lv3:Consoles ; lv4: Consoles;

In [ ]:
rawdata1 = rawdata1.drop_duplicates()
rawdata1 = rawdata1.drop_duplicates(['LEVEL3','LEVEL4'])
rawdata1.head()

,LEVEL1,LEVEL2,LEVEL3,LEVEL4,url,product_cnt,modelling
0,Appliances,Appliance Warranties,Appliance Warranties,Appliance Warranties,https://www.amazon.com/Best-Sellers-Appliances...,1,O
1,Appliances,Cooktops,Cooktops,Cooktops,https://www.amazon.com/Best-Sellers-Appliances...,100,O
2,Appliances,Dishwashers,Built-In Dishwashers,Built-In Dishwashers,https://www.amazon.com/Best-Sellers-Appliances...,96,O
3,Appliances,Dishwashers,Countertop Dishwashers,Countertop Dishwashers,https://www.amazon.com/Best-Sellers-Appliances...,96,O
4,Appliances,Dishwashers,Portable Dishwashers,Portable Dishwashers,https://www.amazon.com/Best-Sellers-Appliances...,56,O


In [ ]:
len(rawdata1)

10630

In [ ]:
df_lv3 = rawdata1.groupby('LEVEL3')['LEVEL4'].apply(', '.join)
df_lv3.head()

LEVEL3
3-D Puzzles                                                             3-D Puzzles
3D Printer Parts & Accessories    3D Printer Accessories, 3D Printer Controllers...
3D Printers                                                             3D Printers
3D Printing Pens                                                   3D Printing Pens
3D Printing Supplies              3D Printing Filament, 3D Printing Granules, 3D...
Name: LEVEL4, dtype: object

In [ ]:
len(df_lv3)

2190

In [ ]:
df_lv3 = pd.merge(rawdata1,df_lv3, on='LEVEL3')
df_lv3_1 = df_lv3.drop_duplicates(['LEVEL3'])
df_lv3_1.tail()

,LEVEL1,LEVEL2,LEVEL3,LEVEL4_x,url,product_cnt,modelling,LEVEL4_y
10624,Toys & Games,Stuffed Animals & Toys,Plush Pillows,Plush Pillows,https://www.amazon.com/Best-Sellers-Toys-Games...,96,O,Plush Pillows
10625,Toys & Games,Stuffed Animals & Toys,Plush Purses,Plush Purses,https://www.amazon.com/Best-Sellers-Toys-Games...,96,O,Plush Purses
10626,Toys & Games,Stuffed Animals & Toys,Plush Toy & Blanket Sets,Plush Toy & Blanket Sets,https://www.amazon.com/Best-Sellers-Toys-Games...,100,O,Plush Toy & Blanket Sets
10627,Toys & Games,Stuffed Animals & Toys,Stuffed Animal Clothing & Accessories,Accessories,https://www.amazon.com/Best-Sellers-Toys-Games...,12,O,"Accessories, Clothing"
10629,Toys & Games,Stuffed Animals & Toys,Stuffed Animals & Teddy Bears,Stuffed Animals & Teddy Bears,https://www.amazon.com/Best-Sellers-Toys-Games...,100,O,Stuffed Animals & Teddy Bears


In [ ]:
len(df_lv3_1)

2190

In [ ]:
#instruction #context #response #category

task2_1_data = pd.DataFrame()

# Generate 'context' column
task2_1_data['context'] = 'level 1:___ '  \
                        + ' level 2:___ '  \
                        + ' level 3: ' + df_lv3_1['LEVEL3'] \
                        + ' level 4:___'


# Generate 'response' column
task2_1_data['response'] = 'level 1: ' + df_lv3_1['LEVEL1'] \
                        + ' level 2: ' + df_lv3_1['LEVEL2'] \
                        + ' level 3: ' + df_lv3_1['LEVEL3'] \
                        + ' level 4: ' + df_lv3_1['LEVEL4_y']


task2_1_data['instruction'] = "Predict higher-level categories and generate lower-level categories using the provided values." #"주어진 값을 이용하여 상위 카테고리를 예측하고 하위 카테고리를 생성하시오."
task2_1_data['category'] = "task2-1"

task2_1_data['index'] = task2_1_data.index

task2_1_data['data'] = "amazon1"
task2_1_data['product_key'] = "category2"


task2_1_data = task2_1_data[['instruction','context','response','category','index','data','product_key']]

task2_1_data.head()

,instruction,context,response,category,index,data,product_key
0,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3: Appliance W...,level 1: Appliances level 2: Appliance Warrant...,task2-1,0,amazon1,category2
1,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3: Cooktops le...,level 1: Appliances level 2: Cooktops level 3:...,task2-1,1,amazon1,category2
2,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3: Built-In Di...,level 1: Appliances level 2: Dishwashers level...,task2-1,2,amazon1,category2
3,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3: Countertop ...,level 1: Appliances level 2: Dishwashers level...,task2-1,3,amazon1,category2
4,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3: Portable Di...,level 1: Appliances level 2: Dishwashers level...,task2-1,4,amazon1,category2


In [ ]:
len(task2_1_data)

2190

## Task 2-2 - 상위, 하위 카테고리 예측 (Lv4)

In [ ]:
#instruction #context #response #category

task2_2_data = pd.DataFrame()

# Generate 'context' column
task2_2_data['context'] = 'level 1:___ '  \
                        + ' level 2:___ '  \
                        + ' level 3:___'  \
                        + ' level 4: ' + df_lv3['LEVEL4_x']

# Generate 'response' column
task2_2_data['response'] = 'level 1: ' + df_lv3['LEVEL1'] \
                        + ' level 2: ' + df_lv3['LEVEL2'] \
                        + ' level 3: ' + df_lv3['LEVEL3'] \
                        + ' level 4: ' + df_lv3['LEVEL4_x']

task2_2_data['instruction'] = "Predict higher-level categories and generate lower-level categories using the provided values."
task2_2_data['category'] = "task2-2"

task2_2_data['index'] = task2_2_data.index

task2_2_data['data'] = "amazon1"


task2_2_data['product_key'] = "category3"



task2_2_data = task2_2_data[['instruction','context','response','category','index','data', 'product_key']]

task2_2_data.head()

,instruction,context,response,category,index,data,product_key
0,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3:___ level 4:...,level 1: Appliances level 2: Appliance Warrant...,task2-2,0,amazon1,category3
1,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3:___ level 4:...,level 1: Appliances level 2: Cooktops level 3:...,task2-2,1,amazon1,category3
2,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3:___ level 4:...,level 1: Appliances level 2: Dishwashers level...,task2-2,2,amazon1,category3
3,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3:___ level 4:...,level 1: Appliances level 2: Dishwashers level...,task2-2,3,amazon1,category3
4,Predict higher-level categories and generate l...,level 1:___ level 2:___ level 3:___ level 4:...,level 1: Appliances level 2: Dishwashers level...,task2-2,4,amazon1,category3


In [ ]:
len(task2_2_data)

10630

## Task 3 - 제품 상세 설명을 기반으로 상품 이름 생성하기

### 예시  
**instruction**  

Generate the name of the product based on the following product detailed description.

<s>다음 제품 상세 설명을 기반으로 상품의 이름을 생성하세요.</s>

**input**  
[detail_table_key][0] is [detail_table_value][0], … , and [detail_table_key][n] is [detail_table_value][n].

**output**  
Blood Pressure Monitors

In [ ]:

df_details = rawdata2[3]
df_details['context']= df_details['detail_table_key'].astype(str) +' is '\
                     + df_details['detail_table_value'].astype(str)
df_details.head()


,url,product_key,product_idx,create_date,detail_table_key,detail_table_value,context
0,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,1,2022-12-02,Product Dimensions,3.5 x 3.8 x 3.7 inches; 8.78 Ounces,Product Dimensions is 3.5 x 3.8 x 3.7 inches; ...
1,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,2,2022-12-02,Item model number,W1681,Item model number is W1681
2,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,3,2022-12-02,Date First Available,2019-09-17 00:00:00,Date First Available is 2019-09-17 00:00:00
3,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,4,2022-12-02,Manufacturer,MMIZOO,Manufacturer is MMIZOO
4,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,5,2022-12-02,ASIN,B07XYW52VQ,ASIN is B07XYW52VQ


In [ ]:
len(df_details)

6475

In [ ]:
context = df_details
context = context.groupby('product_key')['context'].apply(', '.join)

details = pd.merge(rawdata2[3], context, on='product_key')
details = pd.merge(details,rawdata2[5],on='product_key')
details = details.drop_duplicates(['context_y'])
details.tail(10)

,url_x,product_key,product_idx_x,create_date_x,detail_table_key,detail_table_value,context_x,context_y,url_y,product_idx_y,...,level4,product_name,product_price,review_score,review_number,5star,4star,3star,2star,1star
6390,https://www.amazon.com/Best-Sellers-Automotive...,B08QHJX99N,1,2022-12-10,Package Dimensions,12.05 x 9.96 x 4.88 inches,Package Dimensions is 12.05 x 9.96 x 4.88 inches,Package Dimensions is 12.05 x 9.96 x 4.88 inch...,https://www.amazon.com/Best-Sellers-Automotive...,86,...,Battery Chargers,"LZFAN 48V 15 AMP Golf Cart Charger, Golf Cart ...",276.64,4.3,29,68,20,13,0,0
6397,https://www.amazon.com/Best-Sellers-Automotive...,B08VR9BKPC,1,2022-12-10,Package Dimensions,12.32 x 9.41 x 4.65 inches,Package Dimensions is 12.32 x 9.41 x 4.65 inches,Package Dimensions is 12.32 x 9.41 x 4.65 inch...,https://www.amazon.com/Best-Sellers-Automotive...,87,...,Battery Chargers,LZFAN 48V 15A Golf Cart Battery Charger for 48...,276.64,4.0,55,61,11,6,10,13
6404,https://www.amazon.com/Best-Sellers-Automotive...,B09R3XFLNK,1,2022-12-10,Manufacturer,MICTUNING,Manufacturer is MICTUNING,"Manufacturer is MICTUNING, Brand is MICTUNING,...",https://www.amazon.com/Best-Sellers-Automotive...,88,...,Battery Chargers,MICTUNING Motorcycle USB Charger SAE to USB Ad...,13.99,4.8,10,76,24,0,0,0
6413,https://www.amazon.com/Best-Sellers-Automotive...,B0B9H7LNJV,1,2022-12-10,Package Dimensions,12 x 9 x 5 inches,Package Dimensions is 12 x 9 x 5 inches,"Package Dimensions is 12 x 9 x 5 inches, Item ...",https://www.amazon.com/Best-Sellers-Automotive...,89,...,Battery Chargers,Byjoph 48V 15AMP Golf Carts Battery Charger fo...,218.99,0.0,0,0,0,0,0,0
6419,https://www.amazon.com/Best-Sellers-Automotive...,B0742J2BLX,1,2022-12-10,Manufacturer,BatteryMINDer,Manufacturer is BatteryMINDer,"Manufacturer is BatteryMINDer, Brand is Batter...",https://www.amazon.com/Best-Sellers-Automotive...,90,...,Battery Chargers,BatteryMINDer 128CEC2-AA-S5: 12 Volt-8 Amp Con...,\N,4.8,48,83,12,4,0,0
6431,https://www.amazon.com/Best-Sellers-Automotive...,B01GEZA1TO,1,2022-12-10,Manufacturer,XS Power,Manufacturer is XS Power,"Manufacturer is XS Power, Brand is XS Power, M...",https://www.amazon.com/Best-Sellers-Automotive...,91,...,Battery Chargers,"XS Power Li1215 12 Volt, 15 AMP Lithium Ion Ba...",219.99,3.8,3,41,59,0,0,0
6443,https://www.amazon.com/Best-Sellers-Automotive...,B08HRT9RYD,1,2022-12-10,Package Dimensions,13.58 x 12.01 x 7.24 inches,Package Dimensions is 13.58 x 12.01 x 7.24 inches,Package Dimensions is 13.58 x 12.01 x 7.24 inc...,https://www.amazon.com/Best-Sellers-Automotive...,92,...,Battery Chargers,"BMK Car Battery Charger, 12V/24V 5/10/20A Full...",89.99,3.9,95,58,13,9,6,15
6450,https://www.amazon.com/Best-Sellers-Automotive...,B09B1ZFGG7,1,2022-12-10,Package Dimensions,12.2 x 9.49 x 4.72 inches,Package Dimensions is 12.2 x 9.49 x 4.72 inches,Package Dimensions is 12.2 x 9.49 x 4.72 inche...,https://www.amazon.com/Best-Sellers-Automotive...,93,...,Battery Chargers,CRAFTSMAN GOLF 15 AMP 48 Volt Golf Carts Batte...,299.99,3.8,32,57,15,8,20,0
6458,https://www.amazon.com/Best-Sellers-Automotive...,B09V5BF75R,1,2022-12-10,Product Dimensions,7.09 x 3.54 x 2.48 inches,Product Dimensions is 7.09 x 3.54 x 2.48 inches,Product Dimensions is 7.09 x 3.54 x 2.48 inche...,https://www.amazon.com/Best-Sellers-Automotive...,94,...,Battery Chargers,67.2V 5A Charger 60V Smart Charger Charged Aut...,65,4.6,6,58,42,0,0,0
6467,https://www.amazon.com/Best-Sellers-Automotive...,B0742FTCKY,1,2022-12-10,Product Dimensions,4.9 x 7.3 x 2.1 inches,Product Dimensions is 4.9 x 7.3 x 2.1 inches,"Product Dimensions is 4.9 x 7.3 x 2.1 inches, ...",https://www.amazon.com/Best-Sellers-Automotive...,95,...,Battery Chargers,BatteryMINDer 244CEC2-AA-S5: 24 Volt-4 Amp Con...,\N,5.0,31,100,0,0,0,0


In [ ]:
len(details)

584

In [ ]:
#instruction #context #response #category

task3_data = pd.DataFrame()

# Generate 'context' column
task3_data['context'] = details['context_y']

# Generate 'response' column
task3_data['response'] = details['product_name']


task3_data['instruction'] = "Generate the name of the product based on the following product detailed description." #"다음 제품 상세 설명을 기반으로 상품의 이름을 생성하세요."
task3_data['category'] = "task3"

task3_data['index'] = task3_data.index

task3_data['data'] = "amazon2-3"

task3_data['product_key'] = details['product_key']


task3_data = task3_data[['instruction','context','response','category','index','data','product_key']]

task3_data.tail()

,instruction,context,response,category,index,data,product_key
6431,Generate the name of the product based on the ...,"Manufacturer is XS Power, Brand is XS Power, M...","XS Power Li1215 12 Volt, 15 AMP Lithium Ion Ba...",task3,6431,amazon2-3,B01GEZA1TO
6443,Generate the name of the product based on the ...,Package Dimensions is 13.58 x 12.01 x 7.24 inc...,"BMK Car Battery Charger, 12V/24V 5/10/20A Full...",task3,6443,amazon2-3,B08HRT9RYD
6450,Generate the name of the product based on the ...,Package Dimensions is 12.2 x 9.49 x 4.72 inche...,CRAFTSMAN GOLF 15 AMP 48 Volt Golf Carts Batte...,task3,6450,amazon2-3,B09B1ZFGG7
6458,Generate the name of the product based on the ...,Product Dimensions is 7.09 x 3.54 x 2.48 inche...,67.2V 5A Charger 60V Smart Charger Charged Aut...,task3,6458,amazon2-3,B09V5BF75R
6467,Generate the name of the product based on the ...,"Product Dimensions is 4.9 x 7.3 x 2.1 inches, ...",BatteryMINDer 244CEC2-AA-S5: 24 Volt-4 Amp Con...,task3,6467,amazon2-3,B0742FTCKY


## Task 4 - 제품 상세 설명을 기반으로 카테고리 생성하기





### 예시  
**instruction**  
Generate a four-level category for the product based on the following product description.  
<s>다음 제품 상세 설명을 기반으로 상품의 4단계 카테고리를 생성하세요</s>

**input**  
[detail_table_key][0] is [detail_table_value][0], … , and [detail_table_key][n] is [detail_table_value][n].

**output**  
lv1: Health & Household; lv2: Medical Supplies & Equipment; lv3:Health Monitors ; lv4: Blood Pressure Monitors;

In [ ]:
#instruction #context #response #category

task4_data = pd.DataFrame()

# Generate 'context' column
task4_data['context'] = details['context_y']


# Generate 'response' column
task4_data['response'] = 'level 1: ' + details['level1']\
                        + ' level 2: ' + details['level2']\
                        + ' level 3: ' + details['level3'] \
                        + ' level 4: ' + details['level4']


task4_data['instruction'] = "Generate a four-level category for the product based on the following product description." #"다음 제품 상세 설명을 기반으로 상품의 4단계 카테고리를 생성하세요" # ""
task4_data['category'] = "task4"

task4_data['index'] = task4_data.index

task4_data['data'] = "amazon2-3"

task4_data['product_key'] = details['product_key']



task4_data = task4_data[['instruction','context','response','category','index','data','product_key']]

task4_data.tail()

,instruction,context,response,category,index,data,product_key
6431,Generate a four-level category for the product...,"Manufacturer is XS Power, Brand is XS Power, M...",level 1: Automotive level 2: Motorcycle & ATV ...,task4,6431,amazon2-3,B01GEZA1TO
6443,Generate a four-level category for the product...,Package Dimensions is 13.58 x 12.01 x 7.24 inc...,level 1: Automotive level 2: Motorcycle & ATV ...,task4,6443,amazon2-3,B08HRT9RYD
6450,Generate a four-level category for the product...,Package Dimensions is 12.2 x 9.49 x 4.72 inche...,level 1: Automotive level 2: Motorcycle & ATV ...,task4,6450,amazon2-3,B09B1ZFGG7
6458,Generate a four-level category for the product...,Product Dimensions is 7.09 x 3.54 x 2.48 inche...,level 1: Automotive level 2: Motorcycle & ATV ...,task4,6458,amazon2-3,B09V5BF75R
6467,Generate a four-level category for the product...,"Product Dimensions is 4.9 x 7.3 x 2.1 inches, ...",level 1: Automotive level 2: Motorcycle & ATV ...,task4,6467,amazon2-3,B0742FTCKY


In [ ]:
len(task4_data)

584

## Task 5 - 제품 특징 키워드로 카테고리 생성하기

---



### 예시  
**instruction**  
Generate categories based on the following product feature keywords.  
<s>다음 제품 특징 키워드를 기반으로 카테고리를 생성하세요.</s>

**input**  
"Following product feature keywords are are Durability, Easy to read, Easy to use ... Accuracy.  
<s>이 제품의 특징 키워드는 Durability, Easy to read, Easy to use ... Accuracy.</s>

**output**  
lv1: Health & Household; lv2: Medical Supplies & Equipment; lv3:Health Monitors ; lv4: Blood Pressure Monitors;

In [ ]:
df1= rawdata2[2]
df1 = df1.groupby('product_key')['feature_title'].apply(', '.join)
df1.head()

product_key
B0000Y3CRY    Sheerness, Value for money, Scent, Moisturizin...
B0006FMK98                        Scent, Softness, Moisturizing
B0007CXXYG    Moisturizing, Scent, Value for money, For trav...
B0009KN8UA                    Value for money, Longevity, Scent
B0009OMNKM    Scent, Value for money, Light weight, Moisturi...
Name: feature_title, dtype: object

In [ ]:
rawdata2[2].head()

,url,product_key,product_idx,create_date,feature_title,feature_rating
0,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,1,2022-12-02,Durability,4.5
1,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,2,2022-12-02,Easy to read,4.5
2,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,3,2022-12-02,Easy to use,4.4
3,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,4,2022-12-02,Value for money,4.2
4,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,5,2022-12-02,Accuracy,3.9


In [ ]:
len(df1)

497

In [ ]:
len(rawdata2[5])

584

In [ ]:
df1 = pd.merge(rawdata2[2],df1, on='product_key')

In [ ]:
df2 = pd.merge(df1,rawdata2[5], on='product_key')
df2.head()

,url_x,product_key,product_idx_x,create_date_x,feature_title_x,feature_rating,feature_title_y,url_y,product_idx_y,create_date_y,...,level4,product_name,product_price,review_score,review_number,5star,4star,3star,2star,1star
0,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,1,2022-12-02,Durability,4.5,"Durability, Easy to read, Easy to use, Value f...",https://www.amazon.com/Best-Sellers-Health-Hou...,1,2022-12-02,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
1,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,2,2022-12-02,Easy to read,4.5,"Durability, Easy to read, Easy to use, Value f...",https://www.amazon.com/Best-Sellers-Health-Hou...,1,2022-12-02,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
2,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,3,2022-12-02,Easy to use,4.4,"Durability, Easy to read, Easy to use, Value f...",https://www.amazon.com/Best-Sellers-Health-Hou...,1,2022-12-02,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
3,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,4,2022-12-02,Value for money,4.2,"Durability, Easy to read, Easy to use, Value f...",https://www.amazon.com/Best-Sellers-Health-Hou...,1,2022-12-02,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
4,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,5,2022-12-02,Accuracy,3.9,"Durability, Easy to read, Easy to use, Value f...",https://www.amazon.com/Best-Sellers-Health-Hou...,1,2022-12-02,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8


In [ ]:
df2= df2.drop_duplicates(['product_key','feature_title_y'])

In [ ]:
df2.tail()

,url_x,product_key,product_idx_x,create_date_x,feature_title_x,feature_rating,feature_title_y,url_y,product_idx_y,create_date_y,...,level4,product_name,product_price,review_score,review_number,5star,4star,3star,2star,1star
2218,https://www.amazon.com/Best-Sellers-Automotive...,B08QHJX99N,1,2022-12-10,Light weight,4.8,"Light weight, Easy to use, Value for money",https://www.amazon.com/Best-Sellers-Automotive...,86,2022-12-10,...,Battery Chargers,"LZFAN 48V 15 AMP Golf Cart Charger, Golf Cart ...",276.64,4.3,29,68,20,13,0,0
2221,https://www.amazon.com/Best-Sellers-Automotive...,B08VR9BKPC,1,2022-12-10,Easy to install,5.0,"Easy to install, Easy to use, Light weight, Va...",https://www.amazon.com/Best-Sellers-Automotive...,87,2022-12-10,...,Battery Chargers,LZFAN 48V 15A Golf Cart Battery Charger for 48...,276.64,4.0,55,61,11,6,10,13
2225,https://www.amazon.com/Best-Sellers-Automotive...,B08HRT9RYD,1,2022-12-10,Light weight,4.5,"Light weight, Easy to use, Value for money",https://www.amazon.com/Best-Sellers-Automotive...,92,2022-12-10,...,Battery Chargers,"BMK Car Battery Charger, 12V/24V 5/10/20A Full...",89.99,3.9,95,58,13,9,6,15
2228,https://www.amazon.com/Best-Sellers-Automotive...,B09B1ZFGG7,1,2022-12-10,Light weight,4.8,"Light weight, Easy to use, Value for money",https://www.amazon.com/Best-Sellers-Automotive...,93,2022-12-10,...,Battery Chargers,CRAFTSMAN GOLF 15 AMP 48 Volt Golf Carts Batte...,299.99,3.8,32,57,15,8,20,0
2231,https://www.amazon.com/Best-Sellers-Automotive...,B0742FTCKY,1,2022-12-10,Easy to use,5.0,"Easy to use, Value for money",https://www.amazon.com/Best-Sellers-Automotive...,95,2022-12-10,...,Battery Chargers,BatteryMINDer 244CEC2-AA-S5: 24 Volt-4 Amp Con...,\N,5.0,31,100,0,0,0,0


In [ ]:
len(df2)

497

In [ ]:
#instruction #context #response #category

task5_data = pd.DataFrame()

# Generate 'context' column
task5_data['context'] = "Following product feature keywords are " + df2['feature_title_y'] #"이 제품의 특징 키워드는 "


# Generate 'response' column
task5_data['response'] = 'level 1: ' + df2['level1']\
                        + ' level 2: ' + df2['level2']\
                        + ' level 3: ' + df2['level3'] \
                        + ' level 4: ' + df2['level4']


task5_data['instruction'] = "Generate a four-level category for the product based on the following product feature keywords." #"다음 제품 특징 키워드를 기반으로 카테고리를 생성하세요."
task5_data['category'] = "task5"

task5_data['index'] = task5_data.index

task5_data['data'] = "amazon2-2,5"

task5_data['product_key'] = df2['product_key']


task5_data = task5_data[['instruction','context','response','category','index','data','product_key']]

task5_data.tail()

,instruction,context,response,category,index,data,product_key
2218,Generate a four-level category for the product...,Following product feature keywords are Light w...,level 1: Automotive level 2: Motorcycle & ATV ...,task5,2218,"amazon2-2,5",B08QHJX99N
2221,Generate a four-level category for the product...,Following product feature keywords are Easy to...,level 1: Automotive level 2: Motorcycle & ATV ...,task5,2221,"amazon2-2,5",B08VR9BKPC
2225,Generate a four-level category for the product...,Following product feature keywords are Light w...,level 1: Automotive level 2: Motorcycle & ATV ...,task5,2225,"amazon2-2,5",B08HRT9RYD
2228,Generate a four-level category for the product...,Following product feature keywords are Light w...,level 1: Automotive level 2: Motorcycle & ATV ...,task5,2228,"amazon2-2,5",B09B1ZFGG7
2231,Generate a four-level category for the product...,Following product feature keywords are Easy to...,level 1: Automotive level 2: Motorcycle & ATV ...,task5,2231,"amazon2-2,5",B0742FTCKY


## Task 6 - 제품 특징 설명을 기반으로 카테고리 생성하기

### 예시  
**instruction**  
Generate categories based on the following product feature descriptions.  
<s>다음 제품 특징 설명을 기반으로 카테고리를 생성하세요.</s>

**input**  
1. Easy and Fast Reading: Easy to read with a lar...
2. Dual users: Support 2 users and 99 measurement...
3. ...

**output**  
lv1: Health & Household; lv2: Medical Supplies & Equipment; lv3:Health Monitors ; lv4: Blood Pressure Monitors;

In [ ]:
rawdata2[4].head()

,url,product_key,product_idx,create_date,feature_list_content,Unnamed: 5,Unnamed: 6
0,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,1,2022-12-02,Easy and Fast Reading: Easy to read with a lar...,NaN,NaN
1,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,2,2022-12-02,Dual users: Support 2 users and 99 measurement...,NaN,NaN
2,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,3,2022-12-02,Accurate & Well-made: Each blood pressure cuff...,NaN,NaN
3,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,4,2022-12-02,High Blood Pressure Alerter: If your blood pre...,NaN,NaN
4,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,5,2022-12-02,Package Include: Wrist Blood Pressure Monitor*...,NaN,NaN


In [ ]:
len(rawdata2[4])

3253

In [ ]:
content = rawdata2[4]
content['feature_list_content'] =  content['feature_list_content'].astype(str)
content = content.groupby('product_key')['feature_list_content'].apply(' , '.join)

content.head()

product_key
B0000Y3CRY    CLINICALLY PROVEN: Clinically proven to contro...
B0006FMK98    Mild shampoo: This fragrance-free shampoo is p...
B0009KN8UA    16-fluid ounce bottle of Neutrogena T/Gel Ther...
B000A3UBLA    Make sure this fits by entering your model num...
B000F4EMP0    PROFESSIONAL HAIR AND SCALP CLEANSER SHAMPOO f...
Name: feature_list_content, dtype: object

In [ ]:
content = pd.merge(rawdata2[4],content, on='product_key')
content.head()

,url,product_key,product_idx,create_date,feature_list_content_x,Unnamed: 5,Unnamed: 6,feature_list_content_y
0,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,1,2022-12-02,Easy and Fast Reading: Easy to read with a lar...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...
1,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,2,2022-12-02,Dual users: Support 2 users and 99 measurement...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...
2,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,3,2022-12-02,Accurate & Well-made: Each blood pressure cuff...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...
3,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,4,2022-12-02,High Blood Pressure Alerter: If your blood pre...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...
4,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,5,2022-12-02,Package Include: Wrist Blood Pressure Monitor*...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...


In [ ]:
len(content)

3253

In [ ]:
features = pd.merge(content, rawdata2[5], on='product_key')
features.head()

,url_x,product_key,product_idx_x,create_date_x,feature_list_content_x,Unnamed: 5,Unnamed: 6,feature_list_content_y,url_y,product_idx_y,...,level4,product_name,product_price,review_score,review_number,5star,4star,3star,2star,1star
0,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,1,2022-12-02,Easy and Fast Reading: Easy to read with a lar...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...,https://www.amazon.com/Best-Sellers-Health-Hou...,1,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
1,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,2,2022-12-02,Dual users: Support 2 users and 99 measurement...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...,https://www.amazon.com/Best-Sellers-Health-Hou...,1,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
2,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,3,2022-12-02,Accurate & Well-made: Each blood pressure cuff...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...,https://www.amazon.com/Best-Sellers-Health-Hou...,1,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
3,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,4,2022-12-02,High Blood Pressure Alerter: If your blood pre...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...,https://www.amazon.com/Best-Sellers-Health-Hou...,1,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8
4,https://www.amazon.com/Best-Sellers-Health-Hou...,B07XYW52VQ,5,2022-12-02,Package Include: Wrist Blood Pressure Monitor*...,NaN,NaN,Easy and Fast Reading: Easy to read with a lar...,https://www.amazon.com/Best-Sellers-Health-Hou...,1,...,Blood Pressure Monitors,Blood Pressure Monitor Digital Wrist Blood Pre...,29.99,4.3,38265,67,15,7,3,8


In [ ]:
features= features.drop_duplicates(['product_key','feature_list_content_y'])


In [ ]:
print(features.shape[0])

547


In [ ]:
#instruction #context #response #category

task6_data = pd.DataFrame()

# Generate 'context' column
task6_data['context'] = features['feature_list_content_y']

# Generate 'response' column
task6_data['response'] = 'level 1: ' + features['level1']\
                        + ' level 2: ' + features['level2']\
                        + ' level 3: ' + features['level3'] \
                        + ' level 4: ' + features['level4']


task6_data['instruction'] = "Generate a four-level category for the product based on the following product feature descriptions." #"다음 제품 특징 설명을 기반으로 카테고리를 생성하세요."
task6_data['category'] = "task6"

task6_data['index'] = task6_data.index

task6_data['data'] = "amazon2-4,5"

task6_data['product_key'] = features['product_key']


task6_data = task6_data[['instruction','context','response','category','index','data','product_key']]

task6_data.head()

,instruction,context,response,category,index,data,product_key
0,Generate a four-level category for the product...,Easy and Fast Reading: Easy to read with a lar...,level 1: Health & Household level 2: Medical S...,task6,0,"amazon2-4,5",B07XYW52VQ
5,Generate a four-level category for the product...,Make sure this fits by entering your model num...,level 1: Health & Household level 2: Medical S...,task6,5,"amazon2-4,5",B077CXGSJW
11,Generate a four-level category for the product...,"With advanced accuracy, an exclusive Omron tec...",level 1: Health & Household level 2: Medical S...,task6,11,"amazon2-4,5",B07S3LNS1L
16,Generate a four-level category for the product...,The OMRON Silver Upper Arm Monitor provides un...,level 1: Health & Household level 2: Medical S...,task6,16,"amazon2-4,5",B07RYBKNC2
22,Generate a four-level category for the product...,The OMRON Bronze Upper Arm Monitor stores 14 t...,level 1: Health & Household level 2: Medical S...,task6,22,"amazon2-4,5",B07S2H3XB9


In [ ]:
len(task6_data)

547

## Task 7 - 제품 특징 설명을 기반으로 제품 특징 키워드 요약하기

### 예시  
**instruction**  
Summarize the product feature keywords based on the following product feature description.  
<s>다음 제품 특징 설명을 기반으로 제품 특징 키워드 요약하세요.</s>

**input**  
1. Easy and Fast Reading: Easy to read with a lar...
2. Dual users: Support 2 users and 99 measurement...
3. ...

**output**  
Following product feature keywords are Durability, Easy to read, Easy to use ... Accuracy.

In [ ]:
features.columns

Index(['url_x', 'product_key', 'product_idx_x', 'create_date_x',
       'feature_list_content_x', 'Unnamed: 5', 'Unnamed: 6',
       'feature_list_content_y', 'url_y', 'product_idx_y', 'create_date_y',
       'level1', 'level2', 'level3', 'level4', 'product_name', 'product_price',
       'review_score', 'review_number', '5star', '4star', '3star', '2star',
       '1star'],
      dtype='object')

In [ ]:
len(features) # 4,5

547

In [ ]:
df2.columns

Index(['url_x', 'product_key', 'product_idx_x', 'create_date_x',
       'feature_title_x', 'feature_rating', 'feature_title_y', 'url_y',
       'product_idx_y', 'create_date_y', 'level1', 'level2', 'level3',
       'level4', 'product_name', 'product_price', 'review_score',
       'review_number', '5star', '4star', '3star', '2star', '1star'],
      dtype='object')

In [ ]:
len(df2) #2,5

497

In [ ]:
ssf = pd.merge(features, df2, on='product_key')
ssf = ssf.drop_duplicates(['product_key','feature_list_content_y','feature_title_y'])

In [ ]:
len(ssf)

460

In [ ]:
#instruction #context #response #category

task7_data = pd.DataFrame()

# Generate 'context' column
task7_data['context'] = ssf['feature_list_content_y']


# Generate 'response' column
task7_data['response'] = "Following product feature keywords are " + ssf['feature_title_y'] #"이 제품의 특징 키워드는 "


task7_data['instruction'] = "Summarize the product feature keywords based on the following product feature description." #"다음 제품 특징 설명을 기반으로 제품 특징 키워드 요약하세요."
task7_data['category'] = "task7"

task7_data['index'] = task7_data.index

task7_data['data'] = "amazon2-2,4,5"

task7_data['product_key'] = ssf['product_key']


task7_data = task7_data[['instruction','context','response','category','index','data','product_key']]

task7_data.head()

,instruction,context,response,category,index,data,product_key
0,Summarize the product feature keywords based o...,Easy and Fast Reading: Easy to read with a lar...,Following product feature keywords are Durabil...,task7,0,"amazon2-2,4,5",B07XYW52VQ
1,Summarize the product feature keywords based o...,Make sure this fits by entering your model num...,Following product feature keywords are Easy to...,task7,1,"amazon2-2,4,5",B077CXGSJW
2,Summarize the product feature keywords based o...,"With advanced accuracy, an exclusive Omron tec...",Following product feature keywords are Easy to...,task7,2,"amazon2-2,4,5",B07S3LNS1L
3,Summarize the product feature keywords based o...,The OMRON Silver Upper Arm Monitor provides un...,Following product feature keywords are Easy to...,task7,3,"amazon2-2,4,5",B07RYBKNC2
4,Summarize the product feature keywords based o...,The OMRON Bronze Upper Arm Monitor stores 14 t...,Following product feature keywords are Durabil...,task7,4,"amazon2-2,4,5",B07S2H3XB9


In [ ]:
appended_table = pd.concat([task1_data,task2_1_data,task2_2_data,task3_data,task4_data,task5_data,task6_data,task7_data], ignore_index=True)


In [ ]:
appended_table

,instruction,context,response,category,index,data,product_key
0,Classify the product into four levels of categ...,The product name is Blood Pressure Monitor Dig...,The following products can be categorized into...,task1,0,amazon2-5,category1
1,Classify the product into four levels of categ...,The product name is Konquest KBP-2704A Automat...,The following products can be categorized into...,task1,1,amazon2-5,category1
2,Classify the product into four levels of categ...,The product name is Omron Upper Arm Blood Pres...,The following products can be categorized into...,task1,2,amazon2-5,category1
3,Classify the product into four levels of categ...,The product name is OMRON Silver Blood Pressur...,The following products can be categorized into...,task1,3,amazon2-5,category1
4,Classify the product into four levels of categ...,The product name is OMRON Bronze Blood Pressur...,The following products can be categorized into...,task1,4,amazon2-5,category1
...,...,...,...,...,...,...,...
16071,Summarize the product feature keywords based o...,Make sure this fits by entering your model num...,Following product feature keywords are Light w...,task7,455,"amazon2-2,4,5",B08QHJX99N
16072,Summarize the product feature keywords based o...,Make sure this fits by entering your model num...,Following product feature keywords are Easy to...,task7,456,"amazon2-2,4,5",B08VR9BKPC
16073,Summarize the product feature keywords based o...,Make sure this fits by entering your model num...,Following product feature keywords are Light w...,task7,457,"amazon2-2,4,5",B08HRT9RYD
16074,Summarize the product feature keywords based o...,Make sure this fits by entering your model num...,Following product feature keywords are Light w...,task7,458,"amazon2-2,4,5",B09B1ZFGG7


In [ ]:
rawdata2[5]['product_key'].nunique()

584

In [ ]:
rawdata2[4]['product_key'].nunique()

547

In [ ]:
rawdata2[2]['product_key'].nunique()

497

In [ ]:
#영어만을 남기기 위해서 한국어제

df = appended_table


def has_korean_except(text, exceptions):
    for char in text:
        if '가' <= char <= '힣' and char not in exceptions:
            return True
    return False

def replace_korean_chars(text):
    text = text.replace('챰', 'ñ')
    text = text.replace('챔', 'è')
    text = text.replace('챕', 'é')
    text = text.replace('창', 'â')
    return text

df['instruction'] = df['instruction'].apply(replace_korean_chars)
df['context'] = df['context'].apply(replace_korean_chars)
df['response'] = df['response'].apply(replace_korean_chars)

exceptions = []


# 한글이 포함된 셀의 위치 저장
korean_cells = []

for i, column in enumerate(df.columns):
    for j, value in enumerate(df[column]):
        if pd.notna(value) and has_korean_except(str(value), exceptions):
            print(value)
            korean_cells.append((i, j))

# 한글이 포함된 셀의 위치 출력
if len(korean_cells) > 0:
    print("한글이 포함된 셀의 위치:")
    for i, j in korean_cells:
        print(f"행: {i}, 열: {j} - 값: {df.iat[i, j]}")
else:
    print("데이터 프레임에 해당 예외 문자를 제외한 한글이 포함되어 있지 않습니다.")


데이터 프레임에 해당 예외 문자를 제외한 한글이 포함되어 있지 않습니다.


In [ ]:
df.to_csv(file_path +"/KISTI/dataset/amazon_task.csv")